## preprocessing function

In [1]:
def preprocess_dataframe(df):
    # df = df.applymap(lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else x)
    # lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else '' if pd.isna(x) else x
    df.columns = df.columns
    for col in df.columns:
        df[col] = df[col].map(lambda x: x.replace(',', '').replace(' ', '').replace('-','').strip()if isinstance(x, str) else '' if pd.isna(x) else x)
     
    return df

## Function to Combine rows data into single 

In [2]:
def combine_columns(df, new_column_name, *columns):
    
    df[new_column_name] = df[list(columns)].astype(str).agg(' '.join, axis=1).str.strip()
    return df
    

# Remove duplicate column

In [3]:
def column_remover(df):
    a = df.columns.tolist()
    if "Date of Birth_x" and "Date of Birth_y" in a:
        df['Date of Birth'] = df['Date of Birth_x'].combine_first(df['Date of Birth_y'])
        df.drop(['Date of Birth_x', 'Date of Birth_y'], axis=1, inplace=True)
    if "Father_Name_x" and "Father_Name_y" in a:
        df['Father_Name'] = df['Father_Name_x'].combine_first(df['Father_Name_y'])
        df.drop(['Father_Name_x', 'Father_Name_y'], axis=1, inplace=True)
    if "Name_x" and "Name_y" in a:
        df['Name'] = df['Name_x'].combine_first(df['Name_y'])
        df.drop(['Name_x', 'Name_y'], axis=1, inplace=True)
    if "Temporary_Address_x" and "Temporary_Address_y" in a:
        df['Name'] = df['Temporary_Address_x'].combine_first(df['Temporary_Address_y'])
        df.drop(['Temporary_Address_x', 'Temporary_Address_y'], axis=1, inplace=True) 
    if "Mobile Number_x" and "Mobile Number_y" in a:
        df['Mobile Number'] = df['Mobile Number_x'].combine_first(df['Mobile Number_y'])
        df.drop(['Mobile Number_x', 'Mobile Number_y'], axis=1, inplace=True) 
    if "Permanent_Address_x" and "Permanent_Address_y" in a:
        df['Permanent_Address'] = df['Permanent_Address_x'].combine_first(df['Permanent_Address_y'])
        df.drop(['Permanent_Address_x', 'Permanent_Address_y'], axis=1, inplace=True) 
    if "Mother Name_x" and "Mother Name_y" in a:
        df['Mother Name'] = df['Mother Name_x'].combine_first(df['Mother Name_y'])
        df.drop(['Mother Name_x', 'Mother Name_y'], axis=1, inplace=True) 
    return df

## function to perform cosine similarities


In [3]:
 # In this case we do not pass the actual table but in merging time we pass the actual table so merging not performed here


def find_best_matches(df1, df2, df1_combined_col, df2_combined_col, df1_key_col, df2_key_col):
    
    # Combine text for TF-IDF vectorization
    # print(df2[df2_combined_col].tolist())
    combined_text = df1[df1_combined_col].tolist() + df2[df2_combined_col].tolist()
    # print(combined_text)
    vectorizer = TfidfVectorizer().fit(combined_text) 
    tfidf_matrix = vectorizer.transform(combined_text)

    # Split the TF-IDF matrix into original and lookup parts
    tfidf_original = tfidf_matrix[:len(df1)]
    tfidf_lookup = tfidf_matrix[len(df1):]

    # Compute cosine similarities
    cosine_similarities = cosine_similarity(tfidf_original, tfidf_lookup)
    # print(cosine_similarities)

    # Find the best matches
    best_matches = cosine_similarities.argmax(axis=1)
    match_scores = cosine_similarities.max(axis=1)
    
    cosine_sim_table = pd.DataFrame(
        cosine_similarities, 
        index=df1[df1_key_col], 
        columns=df2[df2_key_col]
        
    )
    # print(cosine_sim_table)
    mapping_df = pd.DataFrame({
        'Customer Code': df1['Customer Code'],
        
        'Best Match Index': best_matches,
        'Match Score': match_scores
    })

    
    

    return mapping_df

## Function to merge two tables:


In [4]:
# pass first table as merged table from previous section as we need customer code and this is in layout one
def merging_two_tables(df1, df2, mapping_df, df1_key_col, df2_key_col, df2_name_col):
    
    
    # Add the best match names to the mapping DataFrame
    mapping_df['Best Match Name'] = df2.iloc[mapping_df['Best Match Index']].reset_index(drop=True)[df2_name_col]
    mapping_df.head()

    # Merge the mapping results with the original DataFrames
    merged_df_with_matches = pd.merge(mapping_df, df1, on=df1_key_col, how='left')
    final_result = pd.merge(merged_df_with_matches, df2, left_on='Best Match Name', right_on=df2_key_col, how='left')
    return final_result

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
layout_three_voters = pd.read_csv('layout_three_voters.csv')
layout_two_PQR = pd.read_csv('PQR_layout_2.csv', index_col=0)
layout_one_ = pd.read_csv('ABC_layout_1.csv')
layout_four_KLM = pd.read_csv('KLM_layout_4.csv', index_col=0)
layout_five_DOTM = pd.read_csv('layout_5_license.csv')


In [7]:
layout_three_voters = layout_three_voters.rename(columns={"votersName": "Name","votersFatherName": "Father_Name","votersMotherName": "Mother Name","Permanent_Adress(string)":"Permanent_Address"})
layout_four_KLM = layout_four_KLM.rename(columns={"Father Name": "Father_Name"})
layout_one_ = layout_one_.rename(columns={"First Name": "Name","Father Name": "Father_Name","Permanent_Adress":"Permanent_Address"})

In [8]:
copy_layout_one = layout_one_.copy()
copy_layout_two = layout_two_PQR.copy()
copy_layout_three = layout_three_voters.copy()
copy_layout_four = layout_four_KLM.copy()
copy_layout_five = layout_five_DOTM.copy()

In [9]:
copy_layout_two.head()

,Customer_ID,Name,Date of Birth,Father_Name,Gender,PAN_Number
0,1234567890,Ram Thapa,1990-01-01,Ram Bahadur Thapa,Male,ABCDE1234F
1,2345678901,Sita Shrestha,1991-02-02,Hari Prasad Shrestha,Female,FGHIJ5678K
2,3456789012,Hari Gurung,1992-03-03,Gopal Krishna Gurung,Male,LMNOP9012L
3,4567890123,Gita Tamang,1993-04-04,Shyam Lal Tamang,Female,QRSTU3456M
4,5678901234,Mohan Lama,1994-05-05,Krishna Raj Lama,Male,VWXYZ7890N


# Merging table one and two

In [10]:
layout_one_

,Customer Code,Name,Last Name,Date of Birth,National Id,Father_Name,Mother Name,Permanent_Address,Temporary_Address,Mobile Number
0,21216874,Ram Thapa,Thapa,1990-01-01,AB123C,Ram Bahadur Thapa,Laxmi Thapa,"Baluwatar, Kathmandu, Nepal",NaN,1234567890
1,22359363,Sita Shrestha,Shrestha,1991-02-02,DE456F,Hari Prasad Shrestha,Radha Sharma,"Lakeside, Pokhara, Nepal","New Road, Pokhara, Nepal",2345678901
2,33485241,Hari Gurung,Gurung,1992-03-03,GH789I,Gopal Krishna Gurung,Gita Adhikari,"Chitwan National Park, Chitwan, Nepal","Pulchowk, Lalitpur, Nepal",3456789012
3,45475489,Gita Tamang,Tamang,1993-04-04,JK012L,Shyam Lal Tamang,Mina Rai,"Biratnagar, Morang, Nepal","Bagbazar, Kathmandu, Nepal",4567890123
4,56562139,Mohan Lama,Lama,1994-05-05,MN345O,Krishna Raj Lama,Kalpana Karki,"Bharatpur, Chitwan, Nepal",NaN,5678901234
5,67698214,Radha Magar,Magar,1995-06-06,PQ678R,Narayan Kumar Magar,Nima Gurung,"Butwal, Rupandehi, Nepal","Chabahil, Kathmandu, Nepal",6789012345
6,78714635,Krishna Rai,Rai,1996-07-07,ST901U,Govinda Bahadur Rai,Saru Shrestha,"Hetauda, Makwanpur, Nepal","Kumaripati, Lalitpur, Nepal",7890123456
7,89847326,Sarita Sherpa,Sherpa,1997-08-08,VW234X,Shiva Narayan Sherpa,Nanu Maharjan,"Janakpur, Dhanusa, Nepal",NaN,8901234567
8,91953421,Bikash Karki,Karki,1998-09-09,YZ567A,Bhagirath Bahadur Karki,Lhamu Tamang,"Nepalgunj, Banke, Nepal","Boudha, Kathmandu, Nepal",9012345678
9,10203847,Nisha Bhandari,Bhandari,1999-10-10,BC890D,Surya Bahadur Bhandari,Lila KC,"Dharan, Sunsari, Nepal","Kalanki, Kathmandu, Nepal",123456789


In [11]:
copy_layout_one = combine_columns(copy_layout_one, 'combined_one', 'Name', 'Date of Birth','Father_Name')
copy_layout_two = combine_columns(copy_layout_two, 'combined_two', 'Name', 'Date of Birth','Father_Name')
copy_layout_two


,Customer_ID,Name,Date of Birth,Father_Name,Gender,PAN_Number,combined_two
0,1234567890,Ram Thapa,1990-01-01,Ram Bahadur Thapa,Male,ABCDE1234F,Ram Thapa 1990-01-01 Ram Bahadur Thapa
1,2345678901,Sita Shrestha,1991-02-02,Hari Prasad Shrestha,Female,FGHIJ5678K,Sita Shrestha 1991-02-02 Hari Prasad Shrestha
2,3456789012,Hari Gurung,1992-03-03,Gopal Krishna Gurung,Male,LMNOP9012L,Hari Gurung 1992-03-03 Gopal Krishna Gurung
3,4567890123,Gita Tamang,1993-04-04,Shyam Lal Tamang,Female,QRSTU3456M,Gita Tamang 1993-04-04 Shyam Lal Tamang
4,5678901234,Mohan Lama,1994-05-05,Krishna Raj Lama,Male,VWXYZ7890N,Mohan Lama 1994-05-05 Krishna Raj Lama
5,6789012345,Radha Magar,1995-06-06,Narayan Kumar Magar,Female,ABCD1234PQ,Radha Magar 1995-06-06 Narayan Kumar Magar
6,7890123456,Krishna Rai,1996-07-07,Govinda Bahadur Rai,Male,EFGH5678RS,Krishna Rai 1996-07-07 Govinda Bahadur Rai
7,8901234567,Sarita Sherpa,1997-08-08,Shiva Narayan Sherpa,Female,TUVW9012XY,Sarita Sherpa 1997-08-08 Shiva Narayan Sherpa
8,9012345678,Bikash Karki,1998-09-09,Bhagirath Bahadur Karki,Male,ZABC3456DE,Bikash Karki 1998-09-09 Bhagirath Bahadur Karki
9,123456789,Nisha Bhandari,1999-10-10,Surya Bahadur Bhandari,Female,FGHI7890JK,Nisha Bhandari 1999-10-10 Surya Bahadur Bhandari


In [12]:
copy_layout_one = preprocess_dataframe(copy_layout_one)
copy_layout_two = preprocess_dataframe(copy_layout_two)
copy_layout_two.head()


,Customer_ID,Name,Date of Birth,Father_Name,Gender,PAN_Number,combined_two
0,1234567890,RamThapa,19900101,RamBahadurThapa,Male,ABCDE1234F,RamThapa19900101RamBahadurThapa
1,2345678901,SitaShrestha,19910202,HariPrasadShrestha,Female,FGHIJ5678K,SitaShrestha19910202HariPrasadShrestha
2,3456789012,HariGurung,19920303,GopalKrishnaGurung,Male,LMNOP9012L,HariGurung19920303GopalKrishnaGurung
3,4567890123,GitaTamang,19930404,ShyamLalTamang,Female,QRSTU3456M,GitaTamang19930404ShyamLalTamang
4,5678901234,MohanLama,19940505,KrishnaRajLama,Male,VWXYZ7890N,MohanLama19940505KrishnaRajLama


In [14]:
mapping_one_two = find_best_matches(copy_layout_one,copy_layout_two,'combined_one','combined_two','Customer Code','Name' )
mapping_one_two

,Customer Code,Best Match Index,Match Score
0,21216874,0,1.0
1,22359363,1,1.0
2,33485241,2,1.0
3,45475489,3,1.0
4,56562139,4,1.0
5,67698214,5,1.0
6,78714635,6,1.0
7,89847326,7,1.0
8,91953421,8,1.0
9,10203847,9,1.0


In [16]:
final_one_two_ = merging_two_tables(layout_one_, layout_two_PQR,mapping_one_two,'Customer Code', 'Name', 'Name')
final_one_two_

,Customer Code,Best Match Index,Match Score,Best Match Name,Name_x,Last Name,Date of Birth_x,National Id,Father_Name_x,Mother Name,Permanent_Address,Temporary_Address,Mobile Number,Customer_ID,Name_y,Date of Birth_y,Father_Name_y,Gender,PAN_Number
0,21216874,0,1.0,Ram Thapa,Ram Thapa,Thapa,1990-01-01,AB123C,Ram Bahadur Thapa,Laxmi Thapa,"Baluwatar, Kathmandu, Nepal",NaN,1234567890,1234567890,Ram Thapa,1990-01-01,Ram Bahadur Thapa,Male,ABCDE1234F
1,22359363,1,1.0,Sita Shrestha,Sita Shrestha,Shrestha,1991-02-02,DE456F,Hari Prasad Shrestha,Radha Sharma,"Lakeside, Pokhara, Nepal","New Road, Pokhara, Nepal",2345678901,2345678901,Sita Shrestha,1991-02-02,Hari Prasad Shrestha,Female,FGHIJ5678K
2,33485241,2,1.0,Hari Gurung,Hari Gurung,Gurung,1992-03-03,GH789I,Gopal Krishna Gurung,Gita Adhikari,"Chitwan National Park, Chitwan, Nepal","Pulchowk, Lalitpur, Nepal",3456789012,3456789012,Hari Gurung,1992-03-03,Gopal Krishna Gurung,Male,LMNOP9012L
3,45475489,3,1.0,Gita Tamang,Gita Tamang,Tamang,1993-04-04,JK012L,Shyam Lal Tamang,Mina Rai,"Biratnagar, Morang, Nepal","Bagbazar, Kathmandu, Nepal",4567890123,4567890123,Gita Tamang,1993-04-04,Shyam Lal Tamang,Female,QRSTU3456M
4,56562139,4,1.0,Mohan Lama,Mohan Lama,Lama,1994-05-05,MN345O,Krishna Raj Lama,Kalpana Karki,"Bharatpur, Chitwan, Nepal",NaN,5678901234,5678901234,Mohan Lama,1994-05-05,Krishna Raj Lama,Male,VWXYZ7890N
5,67698214,5,1.0,Radha Magar,Radha Magar,Magar,1995-06-06,PQ678R,Narayan Kumar Magar,Nima Gurung,"Butwal, Rupandehi, Nepal","Chabahil, Kathmandu, Nepal",6789012345,6789012345,Radha Magar,1995-06-06,Narayan Kumar Magar,Female,ABCD1234PQ
6,78714635,6,1.0,Krishna Rai,Krishna Rai,Rai,1996-07-07,ST901U,Govinda Bahadur Rai,Saru Shrestha,"Hetauda, Makwanpur, Nepal","Kumaripati, Lalitpur, Nepal",7890123456,7890123456,Krishna Rai,1996-07-07,Govinda Bahadur Rai,Male,EFGH5678RS
7,89847326,7,1.0,Sarita Sherpa,Sarita Sherpa,Sherpa,1997-08-08,VW234X,Shiva Narayan Sherpa,Nanu Maharjan,"Janakpur, Dhanusa, Nepal",NaN,8901234567,8901234567,Sarita Sherpa,1997-08-08,Shiva Narayan Sherpa,Female,TUVW9012XY
8,91953421,8,1.0,Bikash Karki,Bikash Karki,Karki,1998-09-09,YZ567A,Bhagirath Bahadur Karki,Lhamu Tamang,"Nepalgunj, Banke, Nepal","Boudha, Kathmandu, Nepal",9012345678,9012345678,Bikash Karki,1998-09-09,Bhagirath Bahadur Karki,Male,ZABC3456DE
9,10203847,9,1.0,Nisha Bhandari,Nisha Bhandari,Bhandari,1999-10-10,BC890D,Surya Bahadur Bhandari,Lila KC,"Dharan, Sunsari, Nepal","Kalanki, Kathmandu, Nepal",123456789,123456789,Nisha Bhandari,1999-10-10,Surya Bahadur Bhandari,Female,FGHI7890JK


In [16]:
final_one_two_ = column_remover(final_one_two_)
final_one_two_.drop(['Best Match Index', 'Match Score', 'Best Match Name'], axis=1, inplace=True)
# final_one_two_.drop([ 'Best Match Name', 'Best Match Index'], axis=1, inplace=True)
# final_one_two_.drop([ 'Match Score'], axis=1, inplace=True)

final_one_two_.head()

,Customer Code,Last Name,National Id,Mother Name,Permanent_Address,Temporary_Address,Mobile Number,Customer_ID,Gender,PAN_Number,Date of Birth,Father_Name,Name
0,21216874,Thapa,AB123C,Laxmi Thapa,"Baluwatar, Kathmandu, Nepal",NaN,1234567890,1234567890,Male,ABCDE1234F,1990-01-01,Ram Bahadur Thapa,Ram Thapa
1,22359363,Shrestha,DE456F,Radha Sharma,"Lakeside, Pokhara, Nepal","New Road, Pokhara, Nepal",2345678901,2345678901,Female,FGHIJ5678K,1991-02-02,Hari Prasad Shrestha,Sita Shrestha
2,33485241,Gurung,GH789I,Gita Adhikari,"Chitwan National Park, Chitwan, Nepal","Pulchowk, Lalitpur, Nepal",3456789012,3456789012,Male,LMNOP9012L,1992-03-03,Gopal Krishna Gurung,Hari Gurung
3,45475489,Tamang,JK012L,Mina Rai,"Biratnagar, Morang, Nepal","Bagbazar, Kathmandu, Nepal",4567890123,4567890123,Female,QRSTU3456M,1993-04-04,Shyam Lal Tamang,Gita Tamang
4,56562139,Lama,MN345O,Kalpana Karki,"Bharatpur, Chitwan, Nepal",NaN,5678901234,5678901234,Male,VWXYZ7890N,1994-05-05,Krishna Raj Lama,Mohan Lama


In [17]:
copy_final_one_two = final_one_two_.copy()

# Merging table one two and three

In [18]:
layout_three_voters.head()

,votersID,Name,votersAge,Father_Name,Mother Name,SpouseName,Permanent_Address,Gender
0,11116874,Ram Thapa,45,Ram Bahadur Thapa,Laxmi Thapa,Sita Thapa,"Baluwatar, Kathmandu, Nepal",Male
1,22259363,Sita Shrestha,38,Hari Prasad Shrestha,Radha Sharma,Ravi Sharma,"Lakeside, Pokhara, Nepal",Female
2,33385241,Hari Gurung,52,Gopal Krishna Gurung,Gita Adhikari,Maya Adhikari,"Chitwan National Park, Chitwan, Nepal",Male
3,44475489,Gita Tamang,30,Shyam Lal Tamang,Mina Rai,Surya Rai,"Biratnagar, Morang, Nepal",Female
4,55562139,Mohan Lama,27,Krishna Raj Lama,Kalpana Karki,Sarita Karki,"Bharatpur, Chitwan, Nepal",Male


In [19]:
copy_layout_three = combine_columns(copy_layout_three, 'combined_three', 'Name','Father_Name')
copy_final_one_two = combine_columns(copy_final_one_two, 'combined_one_two', 'Name','Father_Name')
copy_layout_three.head()

,votersID,Name,votersAge,Father_Name,Mother Name,SpouseName,Permanent_Address,Gender,combined_three
0,11116874,Ram Thapa,45,Ram Bahadur Thapa,Laxmi Thapa,Sita Thapa,"Baluwatar, Kathmandu, Nepal",Male,Ram Thapa Ram Bahadur Thapa
1,22259363,Sita Shrestha,38,Hari Prasad Shrestha,Radha Sharma,Ravi Sharma,"Lakeside, Pokhara, Nepal",Female,Sita Shrestha Hari Prasad Shrestha
2,33385241,Hari Gurung,52,Gopal Krishna Gurung,Gita Adhikari,Maya Adhikari,"Chitwan National Park, Chitwan, Nepal",Male,Hari Gurung Gopal Krishna Gurung
3,44475489,Gita Tamang,30,Shyam Lal Tamang,Mina Rai,Surya Rai,"Biratnagar, Morang, Nepal",Female,Gita Tamang Shyam Lal Tamang
4,55562139,Mohan Lama,27,Krishna Raj Lama,Kalpana Karki,Sarita Karki,"Bharatpur, Chitwan, Nepal",Male,Mohan Lama Krishna Raj Lama


In [20]:
copy_final_one_two.head()

,Customer Code,Last Name,National Id,Mother Name,Permanent_Address,Temporary_Address,Mobile Number,Customer_ID,Gender,PAN_Number,Date of Birth,Father_Name,Name,combined_one_two
0,21216874,Thapa,AB123C,Laxmi Thapa,"Baluwatar, Kathmandu, Nepal",NaN,1234567890,1234567890,Male,ABCDE1234F,1990-01-01,Ram Bahadur Thapa,Ram Thapa,Ram Thapa Ram Bahadur Thapa
1,22359363,Shrestha,DE456F,Radha Sharma,"Lakeside, Pokhara, Nepal","New Road, Pokhara, Nepal",2345678901,2345678901,Female,FGHIJ5678K,1991-02-02,Hari Prasad Shrestha,Sita Shrestha,Sita Shrestha Hari Prasad Shrestha
2,33485241,Gurung,GH789I,Gita Adhikari,"Chitwan National Park, Chitwan, Nepal","Pulchowk, Lalitpur, Nepal",3456789012,3456789012,Male,LMNOP9012L,1992-03-03,Gopal Krishna Gurung,Hari Gurung,Hari Gurung Gopal Krishna Gurung
3,45475489,Tamang,JK012L,Mina Rai,"Biratnagar, Morang, Nepal","Bagbazar, Kathmandu, Nepal",4567890123,4567890123,Female,QRSTU3456M,1993-04-04,Shyam Lal Tamang,Gita Tamang,Gita Tamang Shyam Lal Tamang
4,56562139,Lama,MN345O,Kalpana Karki,"Bharatpur, Chitwan, Nepal",NaN,5678901234,5678901234,Male,VWXYZ7890N,1994-05-05,Krishna Raj Lama,Mohan Lama,Mohan Lama Krishna Raj Lama


In [21]:
copy_layout_three = preprocess_dataframe(copy_layout_three)
copy_layout_three.head()

,votersID,Name,votersAge,Father_Name,Mother Name,SpouseName,Permanent_Address,Gender,combined_three
0,11116874,RamThapa,45,RamBahadurThapa,LaxmiThapa,SitaThapa,BaluwatarKathmanduNepal,Male,RamThapaRamBahadurThapa
1,22259363,SitaShrestha,38,HariPrasadShrestha,RadhaSharma,RaviSharma,LakesidePokharaNepal,Female,SitaShresthaHariPrasadShrestha
2,33385241,HariGurung,52,GopalKrishnaGurung,GitaAdhikari,MayaAdhikari,ChitwanNationalParkChitwanNepal,Male,HariGurungGopalKrishnaGurung
3,44475489,GitaTamang,30,ShyamLalTamang,MinaRai,SuryaRai,BiratnagarMorangNepal,Female,GitaTamangShyamLalTamang
4,55562139,MohanLama,27,KrishnaRajLama,KalpanaKarki,SaritaKarki,BharatpurChitwanNepal,Male,MohanLamaKrishnaRajLama


In [22]:
copy_final_one_two = preprocess_dataframe(copy_final_one_two)
copy_final_one_two.head()

,Customer Code,Last Name,National Id,Mother Name,Permanent_Address,Temporary_Address,Mobile Number,Customer_ID,Gender,PAN_Number,Date of Birth,Father_Name,Name,combined_one_two
0,21216874,Thapa,AB123C,LaxmiThapa,BaluwatarKathmanduNepal,NaN,1234567890,1234567890,Male,ABCDE1234F,19900101,RamBahadurThapa,RamThapa,RamThapaRamBahadurThapa
1,22359363,Shrestha,DE456F,RadhaSharma,LakesidePokharaNepal,NewRoadPokharaNepal,2345678901,2345678901,Female,FGHIJ5678K,19910202,HariPrasadShrestha,SitaShrestha,SitaShresthaHariPrasadShrestha
2,33485241,Gurung,GH789I,GitaAdhikari,ChitwanNationalParkChitwanNepal,PulchowkLalitpurNepal,3456789012,3456789012,Male,LMNOP9012L,19920303,GopalKrishnaGurung,HariGurung,HariGurungGopalKrishnaGurung
3,45475489,Tamang,JK012L,MinaRai,BiratnagarMorangNepal,BagbazarKathmanduNepal,4567890123,4567890123,Female,QRSTU3456M,19930404,ShyamLalTamang,GitaTamang,GitaTamangShyamLalTamang
4,56562139,Lama,MN345O,KalpanaKarki,BharatpurChitwanNepal,NaN,5678901234,5678901234,Male,VWXYZ7890N,19940505,KrishnaRajLama,MohanLama,MohanLamaKrishnaRajLama


In [23]:
mapping_one_two_three = find_best_matches(copy_final_one_two,copy_layout_three,'combined_one_two','combined_three','Customer Code','Name' )
mapping_one_two_three.head()

,Customer Code,Best Match Index,Match Score
0,21216874,0,1.0
1,22359363,1,1.0
2,33485241,2,1.0
3,45475489,3,1.0
4,56562139,4,1.0


In [24]:
final_one_two_three = merging_two_tables(final_one_two_, layout_three_voters,mapping_one_two_three,'Customer Code', 'Name', 'Name')
final_one_two_three.head()

,Customer Code,Best Match Index,Match Score,Best Match Name,Last Name,National Id,Mother Name_x,Permanent_Address_x,Temporary_Address,Mobile Number,...,Father_Name_x,Name_x,votersID,Name_y,votersAge,Father_Name_y,Mother Name_y,SpouseName,Permanent_Address_y,Gender
0,21216874,0,1.0,Ram Thapa,Thapa,AB123C,Laxmi Thapa,"Baluwatar, Kathmandu, Nepal",NaN,1234567890,...,Ram Bahadur Thapa,Ram Thapa,11116874,Ram Thapa,45,Ram Bahadur Thapa,Laxmi Thapa,Sita Thapa,"Baluwatar, Kathmandu, Nepal",Male
1,22359363,1,1.0,Sita Shrestha,Shrestha,DE456F,Radha Sharma,"Lakeside, Pokhara, Nepal","New Road, Pokhara, Nepal",2345678901,...,Hari Prasad Shrestha,Sita Shrestha,22259363,Sita Shrestha,38,Hari Prasad Shrestha,Radha Sharma,Ravi Sharma,"Lakeside, Pokhara, Nepal",Female
2,33485241,2,1.0,Hari Gurung,Gurung,GH789I,Gita Adhikari,"Chitwan National Park, Chitwan, Nepal","Pulchowk, Lalitpur, Nepal",3456789012,...,Gopal Krishna Gurung,Hari Gurung,33385241,Hari Gurung,52,Gopal Krishna Gurung,Gita Adhikari,Maya Adhikari,"Chitwan National Park, Chitwan, Nepal",Male
3,45475489,3,1.0,Gita Tamang,Tamang,JK012L,Mina Rai,"Biratnagar, Morang, Nepal","Bagbazar, Kathmandu, Nepal",4567890123,...,Shyam Lal Tamang,Gita Tamang,44475489,Gita Tamang,30,Shyam Lal Tamang,Mina Rai,Surya Rai,"Biratnagar, Morang, Nepal",Female
4,56562139,4,1.0,Mohan Lama,Lama,MN345O,Kalpana Karki,"Bharatpur, Chitwan, Nepal",NaN,5678901234,...,Krishna Raj Lama,Mohan Lama,55562139,Mohan Lama,27,Krishna Raj Lama,Kalpana Karki,Sarita Karki,"Bharatpur, Chitwan, Nepal",Male


In [25]:
final_one_two_three = column_remover(final_one_two_three)
final_one_two_three.drop(['Best Match Index', 'Match Score', 'Best Match Name'], axis=1, inplace=True)
# final_one_two_.drop([ 'Best Match Name'], axis=1, inplace=True)

In [26]:
final_one_two_three.head()

,Customer Code,Last Name,National Id,Temporary_Address,Mobile Number,Customer_ID,Gender,PAN_Number,Date of Birth,votersID,votersAge,SpouseName,Gender,Father_Name,Name,Permanent_Address,Mother Name
0,21216874,Thapa,AB123C,NaN,1234567890,1234567890,Male,ABCDE1234F,1990-01-01,11116874,45,Sita Thapa,Male,Ram Bahadur Thapa,Ram Thapa,"Baluwatar, Kathmandu, Nepal",Laxmi Thapa
1,22359363,Shrestha,DE456F,"New Road, Pokhara, Nepal",2345678901,2345678901,Female,FGHIJ5678K,1991-02-02,22259363,38,Ravi Sharma,Female,Hari Prasad Shrestha,Sita Shrestha,"Lakeside, Pokhara, Nepal",Radha Sharma
2,33485241,Gurung,GH789I,"Pulchowk, Lalitpur, Nepal",3456789012,3456789012,Male,LMNOP9012L,1992-03-03,33385241,52,Maya Adhikari,Male,Gopal Krishna Gurung,Hari Gurung,"Chitwan National Park, Chitwan, Nepal",Gita Adhikari
3,45475489,Tamang,JK012L,"Bagbazar, Kathmandu, Nepal",4567890123,4567890123,Female,QRSTU3456M,1993-04-04,44475489,30,Surya Rai,Female,Shyam Lal Tamang,Gita Tamang,"Biratnagar, Morang, Nepal",Mina Rai
4,56562139,Lama,MN345O,NaN,5678901234,5678901234,Male,VWXYZ7890N,1994-05-05,55562139,27,Sarita Karki,Male,Krishna Raj Lama,Mohan Lama,"Bharatpur, Chitwan, Nepal",Kalpana Karki


## Merging table one, two, three and four

In [27]:
copy_final_one_two_three = final_one_two_three.copy()

In [28]:
copy_layout_four.head()

,Name,Mobile Number,Customer ID,SC Number,Date of Birth,Father_Name
0,Ram Thapa,1234567890,3245,001.01.01,1990-01-01,Ram Bahadur Thapa
1,Sita Shrestha,2345678901,3246,001.01.02,1991-02-02,Hari Prasad Shrestha
2,Hari Gurung,3456789012,3247,001.01.03,1992-03-03,Gopal Krishna Gurung
3,Gita Tamang,4567890123,3248,001.01.04,1993-04-04,Shyam Lal Tamang
4,Mohan Lama,5678901234,3249,001.01.05,1994-05-05,Krishna Raj Lama


In [29]:
copy_layout_four = combine_columns(copy_layout_four, 'combined_four', 'Name','Father_Name', 'Date of Birth')
copy_final_one_two_three = combine_columns(copy_final_one_two_three, 'combined_one_two_three', 'Name','Father_Name','Date of Birth')
copy_layout_four.head()

,Name,Mobile Number,Customer ID,SC Number,Date of Birth,Father_Name,combined_four
0,Ram Thapa,1234567890,3245,001.01.01,1990-01-01,Ram Bahadur Thapa,Ram Thapa Ram Bahadur Thapa 1990-01-01
1,Sita Shrestha,2345678901,3246,001.01.02,1991-02-02,Hari Prasad Shrestha,Sita Shrestha Hari Prasad Shrestha 1991-02-02
2,Hari Gurung,3456789012,3247,001.01.03,1992-03-03,Gopal Krishna Gurung,Hari Gurung Gopal Krishna Gurung 1992-03-03
3,Gita Tamang,4567890123,3248,001.01.04,1993-04-04,Shyam Lal Tamang,Gita Tamang Shyam Lal Tamang 1993-04-04
4,Mohan Lama,5678901234,3249,001.01.05,1994-05-05,Krishna Raj Lama,Mohan Lama Krishna Raj Lama 1994-05-05


In [30]:
copy_final_one_two_three.head()

,Customer Code,Last Name,National Id,Temporary_Address,Mobile Number,Customer_ID,Gender,PAN_Number,Date of Birth,votersID,votersAge,SpouseName,Gender,Father_Name,Name,Permanent_Address,Mother Name,combined_one_two_three
0,21216874,Thapa,AB123C,NaN,1234567890,1234567890,Male,ABCDE1234F,1990-01-01,11116874,45,Sita Thapa,Male,Ram Bahadur Thapa,Ram Thapa,"Baluwatar, Kathmandu, Nepal",Laxmi Thapa,Ram Thapa Ram Bahadur Thapa 1990-01-01
1,22359363,Shrestha,DE456F,"New Road, Pokhara, Nepal",2345678901,2345678901,Female,FGHIJ5678K,1991-02-02,22259363,38,Ravi Sharma,Female,Hari Prasad Shrestha,Sita Shrestha,"Lakeside, Pokhara, Nepal",Radha Sharma,Sita Shrestha Hari Prasad Shrestha 1991-02-02
2,33485241,Gurung,GH789I,"Pulchowk, Lalitpur, Nepal",3456789012,3456789012,Male,LMNOP9012L,1992-03-03,33385241,52,Maya Adhikari,Male,Gopal Krishna Gurung,Hari Gurung,"Chitwan National Park, Chitwan, Nepal",Gita Adhikari,Hari Gurung Gopal Krishna Gurung 1992-03-03
3,45475489,Tamang,JK012L,"Bagbazar, Kathmandu, Nepal",4567890123,4567890123,Female,QRSTU3456M,1993-04-04,44475489,30,Surya Rai,Female,Shyam Lal Tamang,Gita Tamang,"Biratnagar, Morang, Nepal",Mina Rai,Gita Tamang Shyam Lal Tamang 1993-04-04
4,56562139,Lama,MN345O,NaN,5678901234,5678901234,Male,VWXYZ7890N,1994-05-05,55562139,27,Sarita Karki,Male,Krishna Raj Lama,Mohan Lama,"Bharatpur, Chitwan, Nepal",Kalpana Karki,Mohan Lama Krishna Raj Lama 1994-05-05


In [31]:
copy_layout_four = preprocess_dataframe(copy_layout_four)
copy_layout_four.head()

,Name,Mobile Number,Customer ID,SC Number,Date of Birth,Father_Name,combined_four
0,RamThapa,1234567890,3245,001.01.01,19900101,RamBahadurThapa,RamThapaRamBahadurThapa19900101
1,SitaShrestha,2345678901,3246,001.01.02,19910202,HariPrasadShrestha,SitaShresthaHariPrasadShrestha19910202
2,HariGurung,3456789012,3247,001.01.03,19920303,GopalKrishnaGurung,HariGurungGopalKrishnaGurung19920303
3,GitaTamang,4567890123,3248,001.01.04,19930404,ShyamLalTamang,GitaTamangShyamLalTamang19930404
4,MohanLama,5678901234,3249,001.01.05,19940505,KrishnaRajLama,MohanLamaKrishnaRajLama19940505


In [32]:
copy_final_one_two_three = preprocess_dataframe(copy_final_one_two_three)
copy_final_one_two_three.head()

,Customer Code,Last Name,National Id,Temporary_Address,Mobile Number,Customer_ID,Gender,PAN_Number,Date of Birth,votersID,votersAge,SpouseName,Gender,Father_Name,Name,Permanent_Address,Mother Name,combined_one_two_three
0,21216874,Thapa,AB123C,NaN,1234567890,1234567890,Male,ABCDE1234F,19900101,11116874,45,SitaThapa,Male,RamBahadurThapa,RamThapa,BaluwatarKathmanduNepal,LaxmiThapa,RamThapaRamBahadurThapa19900101
1,22359363,Shrestha,DE456F,NewRoadPokharaNepal,2345678901,2345678901,Female,FGHIJ5678K,19910202,22259363,38,RaviSharma,Female,HariPrasadShrestha,SitaShrestha,LakesidePokharaNepal,RadhaSharma,SitaShresthaHariPrasadShrestha19910202
2,33485241,Gurung,GH789I,PulchowkLalitpurNepal,3456789012,3456789012,Male,LMNOP9012L,19920303,33385241,52,MayaAdhikari,Male,GopalKrishnaGurung,HariGurung,ChitwanNationalParkChitwanNepal,GitaAdhikari,HariGurungGopalKrishnaGurung19920303
3,45475489,Tamang,JK012L,BagbazarKathmanduNepal,4567890123,4567890123,Female,QRSTU3456M,19930404,44475489,30,SuryaRai,Female,ShyamLalTamang,GitaTamang,BiratnagarMorangNepal,MinaRai,GitaTamangShyamLalTamang19930404
4,56562139,Lama,MN345O,NaN,5678901234,5678901234,Male,VWXYZ7890N,19940505,55562139,27,SaritaKarki,Male,KrishnaRajLama,MohanLama,BharatpurChitwanNepal,KalpanaKarki,MohanLamaKrishnaRajLama19940505


In [33]:
mapping_one_two_three_four = find_best_matches(copy_final_one_two_three,copy_layout_four,'combined_one_two_three','combined_four','Customer Code','Name' )
mapping_one_two_three_four.head()

,Customer Code,Best Match Index,Match Score
0,21216874,0,1.0
1,22359363,1,1.0
2,33485241,2,1.0
3,45475489,3,1.0
4,56562139,4,1.0


In [34]:
final_one_two_three_four = merging_two_tables(final_one_two_three, layout_four_KLM,mapping_one_two_three,'Customer Code', 'Name', 'Name')
final_one_two_three_four.head()

,Customer Code,Best Match Index,Match Score,Best Match Name,Last Name,National Id,Temporary_Address,Mobile Number_x,Customer_ID,Gender,...,Father_Name_x,Name_x,Permanent_Address,Mother Name,Name_y,Mobile Number_y,Customer ID,SC Number,Date of Birth_y,Father_Name_y
0,21216874,0,1.0,Ram Thapa,Thapa,AB123C,NaN,1234567890,1234567890,Male,...,Ram Bahadur Thapa,Ram Thapa,"Baluwatar, Kathmandu, Nepal",Laxmi Thapa,Ram Thapa,1234567890,3245,001.01.01,1990-01-01,Ram Bahadur Thapa
1,22359363,1,1.0,Sita Shrestha,Shrestha,DE456F,"New Road, Pokhara, Nepal",2345678901,2345678901,Female,...,Hari Prasad Shrestha,Sita Shrestha,"Lakeside, Pokhara, Nepal",Radha Sharma,Sita Shrestha,2345678901,3246,001.01.02,1991-02-02,Hari Prasad Shrestha
2,33485241,2,1.0,Hari Gurung,Gurung,GH789I,"Pulchowk, Lalitpur, Nepal",3456789012,3456789012,Male,...,Gopal Krishna Gurung,Hari Gurung,"Chitwan National Park, Chitwan, Nepal",Gita Adhikari,Hari Gurung,3456789012,3247,001.01.03,1992-03-03,Gopal Krishna Gurung
3,45475489,3,1.0,Gita Tamang,Tamang,JK012L,"Bagbazar, Kathmandu, Nepal",4567890123,4567890123,Female,...,Shyam Lal Tamang,Gita Tamang,"Biratnagar, Morang, Nepal",Mina Rai,Gita Tamang,4567890123,3248,001.01.04,1993-04-04,Shyam Lal Tamang
4,56562139,4,1.0,Mohan Lama,Lama,MN345O,NaN,5678901234,5678901234,Male,...,Krishna Raj Lama,Mohan Lama,"Bharatpur, Chitwan, Nepal",Kalpana Karki,Mohan Lama,5678901234,3249,001.01.05,1994-05-05,Krishna Raj Lama


In [35]:
final_one_two_three_four = column_remover(final_one_two_three_four)
final_one_two_three_four.drop(['Best Match Index', 'Match Score', 'Best Match Name'], axis=1, inplace=True)
# final_one_two_.drop([ 'Best Match Name'], axis=1, inplace=True)

In [36]:
final_one_two_three_four.head()

,Customer Code,Last Name,National Id,Temporary_Address,Customer_ID,Gender,PAN_Number,votersID,votersAge,SpouseName,Gender,Permanent_Address,Mother Name,Customer ID,SC Number,Date of Birth,Father_Name,Name,Mobile Number
0,21216874,Thapa,AB123C,NaN,1234567890,Male,ABCDE1234F,11116874,45,Sita Thapa,Male,"Baluwatar, Kathmandu, Nepal",Laxmi Thapa,3245,001.01.01,1990-01-01,Ram Bahadur Thapa,Ram Thapa,1234567890
1,22359363,Shrestha,DE456F,"New Road, Pokhara, Nepal",2345678901,Female,FGHIJ5678K,22259363,38,Ravi Sharma,Female,"Lakeside, Pokhara, Nepal",Radha Sharma,3246,001.01.02,1991-02-02,Hari Prasad Shrestha,Sita Shrestha,2345678901
2,33485241,Gurung,GH789I,"Pulchowk, Lalitpur, Nepal",3456789012,Male,LMNOP9012L,33385241,52,Maya Adhikari,Male,"Chitwan National Park, Chitwan, Nepal",Gita Adhikari,3247,001.01.03,1992-03-03,Gopal Krishna Gurung,Hari Gurung,3456789012
3,45475489,Tamang,JK012L,"Bagbazar, Kathmandu, Nepal",4567890123,Female,QRSTU3456M,44475489,30,Surya Rai,Female,"Biratnagar, Morang, Nepal",Mina Rai,3248,001.01.04,1993-04-04,Shyam Lal Tamang,Gita Tamang,4567890123
4,56562139,Lama,MN345O,NaN,5678901234,Male,VWXYZ7890N,55562139,27,Sarita Karki,Male,"Bharatpur, Chitwan, Nepal",Kalpana Karki,3249,001.01.05,1994-05-05,Krishna Raj Lama,Mohan Lama,5678901234


## Merging table one, two , three, four and five

In [37]:
copy_final_one_two_three_four= final_one_two_three_four.copy()
copy_final_one_two_three_four.head()

,Customer Code,Last Name,National Id,Temporary_Address,Customer_ID,Gender,PAN_Number,votersID,votersAge,SpouseName,Gender,Permanent_Address,Mother Name,Customer ID,SC Number,Date of Birth,Father_Name,Name,Mobile Number
0,21216874,Thapa,AB123C,NaN,1234567890,Male,ABCDE1234F,11116874,45,Sita Thapa,Male,"Baluwatar, Kathmandu, Nepal",Laxmi Thapa,3245,001.01.01,1990-01-01,Ram Bahadur Thapa,Ram Thapa,1234567890
1,22359363,Shrestha,DE456F,"New Road, Pokhara, Nepal",2345678901,Female,FGHIJ5678K,22259363,38,Ravi Sharma,Female,"Lakeside, Pokhara, Nepal",Radha Sharma,3246,001.01.02,1991-02-02,Hari Prasad Shrestha,Sita Shrestha,2345678901
2,33485241,Gurung,GH789I,"Pulchowk, Lalitpur, Nepal",3456789012,Male,LMNOP9012L,33385241,52,Maya Adhikari,Male,"Chitwan National Park, Chitwan, Nepal",Gita Adhikari,3247,001.01.03,1992-03-03,Gopal Krishna Gurung,Hari Gurung,3456789012
3,45475489,Tamang,JK012L,"Bagbazar, Kathmandu, Nepal",4567890123,Female,QRSTU3456M,44475489,30,Surya Rai,Female,"Biratnagar, Morang, Nepal",Mina Rai,3248,001.01.04,1993-04-04,Shyam Lal Tamang,Gita Tamang,4567890123
4,56562139,Lama,MN345O,NaN,5678901234,Male,VWXYZ7890N,55562139,27,Sarita Karki,Male,"Bharatpur, Chitwan, Nepal",Kalpana Karki,3249,001.01.05,1994-05-05,Krishna Raj Lama,Mohan Lama,5678901234


In [38]:
copy_layout_five.head()

,License Number,Blood Group,Name,Mobile Number,Father_Name,Citizenship Number,Permanent_Address,Temporary_Address,Date of Birth
0,15-05-58353205,AB+,Ram Thapa,1234567890,Ram Bahadur Thapa,624-93227-32431/660086,"Baluwatar, Kathmandu, Nepal","Gongabu, Kathmandu, Nepal",1990-01-01
1,21-08-00435579,AB-,Sita Shrestha,2345678901,Hari Prasad Shrestha,747-42087-31417/584714,"Lakeside, Pokhara, Nepal","New Road, Pokhara, Nepal",1991-02-02
2,93-12-35351480,B-,Hari Gurung,3456789012,Gopal Krishna Gurung,389-45382-93886/821590,"Chitwan National Park, Chitwan, Nepal","Pulchowk, Lalitpur, Nepal",1992-03-03
3,65-03-68139881,A-,Gita Tamang,4567890123,Shyam Lal Tamang,571-38785-99733/440035,"Biratnagar, Morang, Nepal","Bagbazar, Kathmandu, Nepal",1993-04-04
4,14-11-40056582,A+,Mohan Lama,5678901234,Krishna Raj Lama,864-17331-40021/961722,"Bharatpur, Chitwan, Nepal","Balkumari, Lalitpur, Nepal",1994-05-05


In [39]:
copy_layout_five = combine_columns(copy_layout_five, 'combined_five', 'Name','Father_Name', 'Date of Birth')
copy_final_one_two_three_four = combine_columns(copy_final_one_two_three_four, 'combined_one_two_three_four', 'Name','Father_Name','Date of Birth')
copy_layout_four.head()

,Name,Mobile Number,Customer ID,SC Number,Date of Birth,Father_Name,combined_four
0,RamThapa,1234567890,3245,001.01.01,19900101,RamBahadurThapa,RamThapaRamBahadurThapa19900101
1,SitaShrestha,2345678901,3246,001.01.02,19910202,HariPrasadShrestha,SitaShresthaHariPrasadShrestha19910202
2,HariGurung,3456789012,3247,001.01.03,19920303,GopalKrishnaGurung,HariGurungGopalKrishnaGurung19920303
3,GitaTamang,4567890123,3248,001.01.04,19930404,ShyamLalTamang,GitaTamangShyamLalTamang19930404
4,MohanLama,5678901234,3249,001.01.05,19940505,KrishnaRajLama,MohanLamaKrishnaRajLama19940505


In [40]:
copy_layout_five = preprocess_dataframe(copy_layout_five)
copy_layout_five.head()

,License Number,Blood Group,Name,Mobile Number,Father_Name,Citizenship Number,Permanent_Address,Temporary_Address,Date of Birth,combined_five
0,150558353205,AB+,RamThapa,1234567890,RamBahadurThapa,6249322732431/660086,BaluwatarKathmanduNepal,GongabuKathmanduNepal,19900101,RamThapaRamBahadurThapa19900101
1,210800435579,AB,SitaShrestha,2345678901,HariPrasadShrestha,7474208731417/584714,LakesidePokharaNepal,NewRoadPokharaNepal,19910202,SitaShresthaHariPrasadShrestha19910202
2,931235351480,B,HariGurung,3456789012,GopalKrishnaGurung,3894538293886/821590,ChitwanNationalParkChitwanNepal,PulchowkLalitpurNepal,19920303,HariGurungGopalKrishnaGurung19920303
3,650368139881,A,GitaTamang,4567890123,ShyamLalTamang,5713878599733/440035,BiratnagarMorangNepal,BagbazarKathmanduNepal,19930404,GitaTamangShyamLalTamang19930404
4,141140056582,A+,MohanLama,5678901234,KrishnaRajLama,8641733140021/961722,BharatpurChitwanNepal,BalkumariLalitpurNepal,19940505,MohanLamaKrishnaRajLama19940505


In [41]:
copy_final_one_two_three_four = preprocess_dataframe(copy_final_one_two_three_four)
copy_final_one_two_three_four.head()

,Customer Code,Last Name,National Id,Temporary_Address,Customer_ID,Gender,PAN_Number,votersID,votersAge,SpouseName,Gender,Permanent_Address,Mother Name,Customer ID,SC Number,Date of Birth,Father_Name,Name,Mobile Number,combined_one_two_three_four
0,21216874,Thapa,AB123C,NaN,1234567890,Male,ABCDE1234F,11116874,45,SitaThapa,Male,BaluwatarKathmanduNepal,LaxmiThapa,3245,001.01.01,19900101,RamBahadurThapa,RamThapa,1234567890,RamThapaRamBahadurThapa19900101
1,22359363,Shrestha,DE456F,NewRoadPokharaNepal,2345678901,Female,FGHIJ5678K,22259363,38,RaviSharma,Female,LakesidePokharaNepal,RadhaSharma,3246,001.01.02,19910202,HariPrasadShrestha,SitaShrestha,2345678901,SitaShresthaHariPrasadShrestha19910202
2,33485241,Gurung,GH789I,PulchowkLalitpurNepal,3456789012,Male,LMNOP9012L,33385241,52,MayaAdhikari,Male,ChitwanNationalParkChitwanNepal,GitaAdhikari,3247,001.01.03,19920303,GopalKrishnaGurung,HariGurung,3456789012,HariGurungGopalKrishnaGurung19920303
3,45475489,Tamang,JK012L,BagbazarKathmanduNepal,4567890123,Female,QRSTU3456M,44475489,30,SuryaRai,Female,BiratnagarMorangNepal,MinaRai,3248,001.01.04,19930404,ShyamLalTamang,GitaTamang,4567890123,GitaTamangShyamLalTamang19930404
4,56562139,Lama,MN345O,NaN,5678901234,Male,VWXYZ7890N,55562139,27,SaritaKarki,Male,BharatpurChitwanNepal,KalpanaKarki,3249,001.01.05,19940505,KrishnaRajLama,MohanLama,5678901234,MohanLamaKrishnaRajLama19940505


In [42]:
mapping_one_two_three_four_five = find_best_matches(copy_final_one_two_three_four,copy_layout_five,'combined_one_two_three_four','combined_five','Customer Code','Name' )
mapping_one_two_three_four_five.head()

,Customer Code,Best Match Index,Match Score
0,21216874,0,1.0
1,22359363,1,1.0
2,33485241,2,1.0
3,45475489,3,1.0
4,56562139,4,1.0


In [43]:
final_one_two_three_four.head()

,Customer Code,Last Name,National Id,Temporary_Address,Customer_ID,Gender,PAN_Number,votersID,votersAge,SpouseName,Gender,Permanent_Address,Mother Name,Customer ID,SC Number,Date of Birth,Father_Name,Name,Mobile Number
0,21216874,Thapa,AB123C,NaN,1234567890,Male,ABCDE1234F,11116874,45,Sita Thapa,Male,"Baluwatar, Kathmandu, Nepal",Laxmi Thapa,3245,001.01.01,1990-01-01,Ram Bahadur Thapa,Ram Thapa,1234567890
1,22359363,Shrestha,DE456F,"New Road, Pokhara, Nepal",2345678901,Female,FGHIJ5678K,22259363,38,Ravi Sharma,Female,"Lakeside, Pokhara, Nepal",Radha Sharma,3246,001.01.02,1991-02-02,Hari Prasad Shrestha,Sita Shrestha,2345678901
2,33485241,Gurung,GH789I,"Pulchowk, Lalitpur, Nepal",3456789012,Male,LMNOP9012L,33385241,52,Maya Adhikari,Male,"Chitwan National Park, Chitwan, Nepal",Gita Adhikari,3247,001.01.03,1992-03-03,Gopal Krishna Gurung,Hari Gurung,3456789012
3,45475489,Tamang,JK012L,"Bagbazar, Kathmandu, Nepal",4567890123,Female,QRSTU3456M,44475489,30,Surya Rai,Female,"Biratnagar, Morang, Nepal",Mina Rai,3248,001.01.04,1993-04-04,Shyam Lal Tamang,Gita Tamang,4567890123
4,56562139,Lama,MN345O,NaN,5678901234,Male,VWXYZ7890N,55562139,27,Sarita Karki,Male,"Bharatpur, Chitwan, Nepal",Kalpana Karki,3249,001.01.05,1994-05-05,Krishna Raj Lama,Mohan Lama,5678901234


In [44]:
final_one_two_three_four_five = merging_two_tables(final_one_two_three_four, layout_five_DOTM, mapping_one_two_three_four_five,'Customer Code', 'Name', 'Name')
(final_one_two_three_four_five.head())

,Customer Code,Best Match Index,Match Score,Best Match Name,Last Name,National Id,Temporary_Address_x,Customer_ID,Gender,PAN_Number,...,Mobile Number_x,License Number,Blood Group,Name_y,Mobile Number_y,Father_Name_y,Citizenship Number,Permanent_Address_y,Temporary_Address_y,Date of Birth_y
0,21216874,0,1.0,Ram Thapa,Thapa,AB123C,NaN,1234567890,Male,ABCDE1234F,...,1234567890,15-05-58353205,AB+,Ram Thapa,1234567890,Ram Bahadur Thapa,624-93227-32431/660086,"Baluwatar, Kathmandu, Nepal","Gongabu, Kathmandu, Nepal",1990-01-01
1,22359363,1,1.0,Sita Shrestha,Shrestha,DE456F,"New Road, Pokhara, Nepal",2345678901,Female,FGHIJ5678K,...,2345678901,21-08-00435579,AB-,Sita Shrestha,2345678901,Hari Prasad Shrestha,747-42087-31417/584714,"Lakeside, Pokhara, Nepal","New Road, Pokhara, Nepal",1991-02-02
2,33485241,2,1.0,Hari Gurung,Gurung,GH789I,"Pulchowk, Lalitpur, Nepal",3456789012,Male,LMNOP9012L,...,3456789012,93-12-35351480,B-,Hari Gurung,3456789012,Gopal Krishna Gurung,389-45382-93886/821590,"Chitwan National Park, Chitwan, Nepal","Pulchowk, Lalitpur, Nepal",1992-03-03
3,45475489,3,1.0,Gita Tamang,Tamang,JK012L,"Bagbazar, Kathmandu, Nepal",4567890123,Female,QRSTU3456M,...,4567890123,65-03-68139881,A-,Gita Tamang,4567890123,Shyam Lal Tamang,571-38785-99733/440035,"Biratnagar, Morang, Nepal","Bagbazar, Kathmandu, Nepal",1993-04-04
4,56562139,4,1.0,Mohan Lama,Lama,MN345O,NaN,5678901234,Male,VWXYZ7890N,...,5678901234,14-11-40056582,A+,Mohan Lama,5678901234,Krishna Raj Lama,864-17331-40021/961722,"Bharatpur, Chitwan, Nepal","Balkumari, Lalitpur, Nepal",1994-05-05


In [45]:
final_one_two_three_four_five = column_remover(final_one_two_three_four_five)
final_one_two_three_four_five.drop(['Best Match Index', 'Match Score', 'Best Match Name'], axis=1, inplace=True)
# final_one_two_.drop([ 'Best Match Name'], axis=1, inplace=True)

In [46]:
final_one_two_three_four_five.head()

,Customer Code,Last Name,National Id,Customer_ID,Gender,PAN_Number,votersID,votersAge,SpouseName,Gender,...,Customer ID,SC Number,License Number,Blood Group,Citizenship Number,Date of Birth,Father_Name,Name,Mobile Number,Permanent_Address
0,21216874,Thapa,AB123C,1234567890,Male,ABCDE1234F,11116874,45,Sita Thapa,Male,...,3245,001.01.01,15-05-58353205,AB+,624-93227-32431/660086,1990-01-01,Ram Bahadur Thapa,"Gongabu, Kathmandu, Nepal",1234567890,"Baluwatar, Kathmandu, Nepal"
1,22359363,Shrestha,DE456F,2345678901,Female,FGHIJ5678K,22259363,38,Ravi Sharma,Female,...,3246,001.01.02,21-08-00435579,AB-,747-42087-31417/584714,1991-02-02,Hari Prasad Shrestha,"New Road, Pokhara, Nepal",2345678901,"Lakeside, Pokhara, Nepal"
2,33485241,Gurung,GH789I,3456789012,Male,LMNOP9012L,33385241,52,Maya Adhikari,Male,...,3247,001.01.03,93-12-35351480,B-,389-45382-93886/821590,1992-03-03,Gopal Krishna Gurung,"Pulchowk, Lalitpur, Nepal",3456789012,"Chitwan National Park, Chitwan, Nepal"
3,45475489,Tamang,JK012L,4567890123,Female,QRSTU3456M,44475489,30,Surya Rai,Female,...,3248,001.01.04,65-03-68139881,A-,571-38785-99733/440035,1993-04-04,Shyam Lal Tamang,"Bagbazar, Kathmandu, Nepal",4567890123,"Biratnagar, Morang, Nepal"
4,56562139,Lama,MN345O,5678901234,Male,VWXYZ7890N,55562139,27,Sarita Karki,Male,...,3249,001.01.05,14-11-40056582,A+,864-17331-40021/961722,1994-05-05,Krishna Raj Lama,"Balkumari, Lalitpur, Nepal",5678901234,"Bharatpur, Chitwan, Nepal"


# Final Expected Table

In [49]:
import numpy as np

In [60]:
def create_final_table(merged_df,start_id=3242305):
    
    columns = ['Customer_Id', 'Name', 'DOB', 'Permanent_Address', 'Mobile_Number', 'Age','Father_Name','Mother_Name','Email','Citizenship_Number']
    individual_info_df = pd.DataFrame(columns=columns)
    
    # Map the columns from merged_df to the new DataFrame
    # Adjust the keys in this dictionary according to the actual column names in your  merged_df
    column_mapping = {
        'Customer_Id': 'Customer_ID',
        'Name': 'Name',
        'DOB': 'Date of Birth',
        'Permanent_Address': 'Permanent_Address',
        'Mobile_Number': 'Mobile Number',
        'Age': 'votersAge',
        'Father_Name': 'Father_Name',
        'Mother_Name': 'Mother Name',
        'Email': 'Email',        
        'Citizenship_Number': 'Citizenship Number',
    }
    
    for new_col, original_col in column_mapping.items():
        
        if original_col in merged_df.columns:
            individual_info_df[new_col] =  merged_df[original_col]
        else:
            individual_info_df[new_col] = np.nan  # Fill with NaN if the column does not exist in  merged_df
    
    
    individual_info_df['universal_id'] = range(start_id, start_id + len(merged_df))
    
    
    return individual_info_df

In [61]:
universal_df = create_final_table(final_one_two_three_four_five, start_id=3242305)
universal_df.head()

,Customer_Id,Name,DOB,Permanent_Address,Mobile_Number,Age,Father_Name,Mother_Name,Email,Citizenship_Number,universal_id
0,1234567890,"Gongabu, Kathmandu, Nepal",1990-01-01,"Baluwatar, Kathmandu, Nepal",1234567890,45,Ram Bahadur Thapa,Laxmi Thapa,NaN,624-93227-32431/660086,3242305
1,2345678901,"New Road, Pokhara, Nepal",1991-02-02,"Lakeside, Pokhara, Nepal",2345678901,38,Hari Prasad Shrestha,Radha Sharma,NaN,747-42087-31417/584714,3242306
2,3456789012,"Pulchowk, Lalitpur, Nepal",1992-03-03,"Chitwan National Park, Chitwan, Nepal",3456789012,52,Gopal Krishna Gurung,Gita Adhikari,NaN,389-45382-93886/821590,3242307
3,4567890123,"Bagbazar, Kathmandu, Nepal",1993-04-04,"Biratnagar, Morang, Nepal",4567890123,30,Shyam Lal Tamang,Mina Rai,NaN,571-38785-99733/440035,3242308
4,5678901234,"Balkumari, Lalitpur, Nepal",1994-05-05,"Bharatpur, Chitwan, Nepal",5678901234,27,Krishna Raj Lama,Kalpana Karki,NaN,864-17331-40021/961722,3242309
